<h3> Bhapkar test in Python</h3>

<p> Matt Wilder, University of Toronto <br>
Please address questions and comments to <a href="mailto:matt.wilder@utoronto.ca">matt.wilder@utoronto.ca</a>. </p>

<p>Updated 25 March 2024</p>

<p>This function runs Bhapkar's test from Bhapkar (1968) 'On the analysis of contingency tables with a quantitative response' <i> Biometrics, 24</i>(2): 329-38.</p>


In [ ]:
import numpy as np
from scipy.stats import chi2

# reproduce the example from Bhakpar (1968) 

def bhapkar_test(n_ij, a_i, b_j):
    N_j = n_ij.sum(axis=0)
    A_i = np.array([np.sum(a_i * n_ij[:, j]) / N_j[j] for j in range(len(N_j))])
    B_i = np.array([np.sum((a_i - A_i[j])**2 * (n_ij[:, j] / N_j[j])) for j in range(len(N_j))])
    w_i = N_j / B_i
    w = np.sum(w_i)
    C = np.sum(w_i * A_i)
    D = np.sum(b_j * w_i)
    E = np.sum(b_j**2 * w_i)
    F = np.sum(b_j * A_i * w_i)
    
    df_H1 = len(b_j) - 1
    df_H2 = len(b_j) - 2
    df_H3 = 1
    
    chi_squared_H1 = np.sum(w_i * A_i**2) - C**2 / w
    
    # calculate X hat and mu hat
    denom = E * w - D**2
    if denom > 0:
        X_hat = (E * C - D * F) / denom
        mu_hat = (w * F - C * D) / denom
        chi_squared_H2 = np.sum(w_i * (A_i - (X_hat + b_j * mu_hat))**2)
        chi_squared_H3 = chi_squared_H1 - chi_squared_H2
    else:
        X_hat = np.nan
        mu_hat = np.nan
        chi_squared_H2 = np.nan
        chi_squared_H3 = np.nan
    
    # calculate p-values for each hypothesis test
    p_value_H1 = chi2.sf(chi_squared_H1, df_H1) if not np.isnan(chi_squared_H1) else np.nan
    p_value_H2 = chi2.sf(chi_squared_H2, df_H2) if not np.isnan(chi_squared_H2) else np.nan
    p_value_H3 = chi2.sf(chi_squared_H3, df_H3) if not np.isnan(chi_squared_H3) else np.nan
    
    return {
        'N_j': N_j, 'A_i': A_i, 'B_i': B_i, 'w_i': w_i, 'w': w, 'C': C, 'D': D, 'E': E, 'F': F,
        'chi_squared_H0': chi_squared_H1, 'df_H0': df_H1, 'p_value_H0': p_value_H1,
        'chi_squared_H1': chi_squared_H2, 'df_H1': df_H2, 'p_value_H1': p_value_H2,
        'chi_squared_H2': chi_squared_H3, 'df_H2': df_H3, 'p_value_H2': p_value_H3,
        'X_hat': X_hat, 'mu_hat': mu_hat
    }

# define the crosstab
n_ij = np.array([
    [141, 67, 114, 79, 39],  # teacher's ratings under different homework conditions from Bhakpar (1968)
    [131, 66, 143, 72, 35],
    [36, 14, 38, 28, 16]
])
a_i = np.array([1, 0, -1])  # scores for the teacher's rating categories from Bhakpar (1968)
b_j = np.array([2, 1, 0, -1, -2])  # homework condition scores Bhakpar from (1968)

# perform the test
results = bhapkar_test(n_ij, a_i, b_j)

# print the results
print(f"H₀ (Homogeneity of mean scores): χ² = {results['chi_squared_H0']:.2f}, df = {results['df_H0']}, p-value = {results['p_value_H0']:.4f}")
print(f"H₁ (Linearity of regression): χ² = {results['chi_squared_H1']:.2f}, df = {results['df_H1']}, p-value = {results['p_value_H1']:.4f}")
print(f"H₂ (Significance of regression coefficient): χ² = {results['chi_squared_H2']:.2f}, df = {results['df_H2']}, p-value = {results['p_value_H2']:.4f}")

# presentation of N_j, A_i, B_i, and w_i
print("\nSample sizes N for each condition j (Nj):", results['N_j'])
for index, (a, b, w) in enumerate(zip(results['A_i'], results['B_i'], results['w_i']), start=1):
    print(f"Group {index}: Mean Score (A{index}) = {a:.2f}, In-group Variability (B{index}) = {b:.2f}, Group Weight (w{index}) = {w:.2f}")

# explanation of variables
print("\nExplanation of variables:")
print("Nj: Sample sizes for each condition (j).")
print("Ai: Group mean scores, where 'i' denotes the group index.")
print("Bi: Measure of in-group variability for each group 'i'.")
print("w: Weight assigned to each group 'i', calculated based on Bi and sample size.")

<b><br>Original results from Bhapkar (1968):</b>

<i>H</i><sub>0</sub> (Homogeneity of mean scores): χ2 = 3.957, df = 4, <i>p</i>-value not provided but stated as insignificant.
<i>H</i><sub>1</sub> (Linearity of regression): χ<sup>2</sup> = 1.578, df = 3, <i>p</i>-value not provided but stated as insignificant.
<i>H</i><sub>2</sub> (Significance of the regression coefficient): χ2 = 2.379, df = 1, <i>p</i>-value not provided but stated as insignificant.
